In [1]:
from keras.models import load_model

C:\Users\wy_cl\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


### load model weights from a keras model

In this function, I am able to take a keras model and read the each layer's weight matrix and configuration. Then I save the each layer's weight matrix in one list so that I could modify them later.

In [9]:
def load_model_weights(model):
    # Create two variables to store the layer information and weight matrixes.
    layers_name = []
    weights = []
    
    # Loop through the model layer by layer
    for layer in model.layers:
        # extract the layer information by using layer.get_config()
        layers_name.append(layer.get_config())
        
        # extract the weight matrix for this layer using layer.get_weights()
        weights.append(layer.get_weights())
    return weights, layers_name

### save the modified model weights into the keras model

In [ ]:
def save_model_weights(model, new_weights):
    # Loop through the list containing each layer's new weight
    for i in range(len(new_weights)):
        # set the weight for each layer using set_weight()
        model.layers[i].set_weights(new_weights[i])
    return model

In [6]:
model_path = "test_model_files/cifar10_model.h5"
model = load_model(model_path)
# print_model_structure(model)
weights, model_struct = load_model_weights(model)

In [10]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
activation_3 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 32, 32, 48)        13872     
__________

In [19]:
weights[0][0].shape

(3, 3, 3, 32)

In [20]:
weights[0][1].shape

(32,)

From the above, using the model.summary() I can see each layer's information and the size. To test if the load_model_weights() is working, I check the shape of first layer's weight matrix. The first layer weights contains two weight matrix, with shape of (3, 3, 3, 32) and (32,), which means there are 896 parameters in the first layer and the number agrees to the first row of table from model.summarty()